In [1]:
#the purpose of this piece of code is to detect and collect connected 
#components given by the encoder-decoder masks of images. And then 
#the code will output bounding boxes of these connected components.

import cv2
import matplotlib.pyplot as plt

class Point:
    def __init__(self,setX,setY):
        self.x = setX
        self.y = setY
    def printPoint(self):
        print("x: "+str(self.x))
        print("y: "+str(self.y))
class BoundingBox:
    def __init__(self,setLogicalClass,setXLeft,setXRight,setYTop,setYBottom):
        self.logicalClass = setLogicalClass
        self.XLeft = setXLeft
        self.XRight = setXRight
        self.YTop = setYTop
        self.YBottom = setYBottom
    def printBox(self):
        print("logicalClass: "+str(self.logicalClass))
        print("x_left: "+str(self.XLeft))
        print("x_right: "+str(self.XRight))
        print("y_top: "+str(self.YTop))
        print("y_bottom: "+str(self.YBottom))
#read image
image_list = []
def read_images_from_folder(folder_path,n,prefix=""):
    image_list = []
    for i in range(n):
        #res_0.png_res.png
        input_file_name = folder_path+prefix+str(i)+".png_res.png"
        image_single = cv2.imread(input_file_name)
        image_list.append(image_single)
    return image_list

input_folder = "./cnnSeg/0_entro/"
n_images = 300
prefix = "res_"

image_list = read_images_from_folder(input_folder,n_images,prefix)


<Figure size 640x480 with 1 Axes>

In [2]:
# detect connected components
# convert to binary images
import numpy as np
def convert_to_binary(image):
    kernel = np.ones((7,7),np.uint8)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,image_binary = cv2.threshold(image_gray,0,255,cv2.THRESH_OTSU)#OTSU thresholding is the ordinary way of binarizing document image
    image_binary = cv2.medianBlur(image_binary, 5)
    image_binary = cv2.erode(image_binary,kernel,iterations = 2)
    image_binary = cv2.dilate(image_binary,kernel,iterations = 2)
    return image_binary

image_binary_list = []


for image in image_list:
    image_binary = convert_to_binary(image)
    image_binary_list.append(image_binary)
    
#connected component detection
contour_list = []
for i in range(len(image_list)):
    image_single = image_binary_list[i]
    contours, hierarchy = cv2.findContours(image_single, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)          
    contour_list.append(contours)
    #output_image = cv2.drawContours(image_list[i], contours, -1, (0,255,0), 3)
    #cv2.imwrite("./acdc/output_images/out_"+str(i)+".png",output_image)

In [4]:
#convert contours to bounding boxes format
def convert_contours_to_bounding_boxes(contours):
    bounding_boxes_list = []
    for i in range(len(contours)):
        x_left = -1
        x_right = -1
        y_top = -1
        y_bottom = -1
        
        contour_single = contours[i]
        for j in range(len(contours[i])):
            x = contour_single[j][0][0]
            y = contour_single[j][0][1]
            
            if(x_left == -1 or x<x_left):
                x_left = x
            if(x_right == -1 or x>x_right):
                x_right = x
            if(y_top == -1 or y<y_top):
                y_top = y
            if(y_bottom == -1 or y>y_bottom):
                y_bottom = y
                
        b_box = BoundingBox(1,x_left,x_right,y_top,y_bottom)
        if(x_left!=x_right and y_top!=y_bottom):
            bounding_boxes_list.append(b_box)
    return bounding_boxes_list

labels_lists = []
bounding_boxes_list = []
for i in range(len(contour_list)):
    bounding_boxes = convert_contours_to_bounding_boxes(contour_list[i])   
#     bounding_boxes = merging_rebounding_boxes(bounding_boxes)
    single_label_list = []
    for bounding_box in bounding_boxes:
        single_label_list.append(bounding_box.logicalClass)
        start_point = (bounding_box.XLeft,bounding_box.YTop)
        end_point = (bounding_box.XRight,bounding_box.YBottom)
        image = cv2.rectangle(image_list[i], start_point, end_point, color=(255,0,0), thickness=3) 
    cv2.imwrite("output_images/"+str(i)+".png",image)
    labels_lists.append(single_label_list)
    bounding_boxes_list.append(bounding_boxes)
    # print(contours[i][0][0][x,y])

In [5]:
#write the results of the bounding boxes found
import csv
def write_box_info_into_csv(boxes_accepted_list,labels_list,saving_root_boxes):
    for i in range(len(boxes_accepted_list)):
        with open(saving_root_boxes+str(i)+'_boxes.csv', 'w', newline='') as csvfile:
            fieldnames = ['xmin', 'ymin','xmax','ymax','label']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for j in range(len(boxes_accepted_list[i])):
                xmin = boxes_accepted_list[i][j].XLeft
                ymin = boxes_accepted_list[i][j].YTop
                xmax = boxes_accepted_list[i][j].XRight
                ymax = boxes_accepted_list[i][j].YBottom
                writer.writerow({'xmin': str(int(xmin)), 'ymin': str(int(ymin)),'xmax': str(int(xmax)),'ymax': str(int(ymax)),'label':str(int(labels_list[i][j]))})

write_box_info_into_csv(bounding_boxes_list,labels_lists,"./bce_csv_text_only/")